In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

测试集B

In [17]:
train = pd.read_csv('../data/train_featureV2.csv')
test = pd.read_csv('../data/test_featureV2.csv')

In [18]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [19]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 64,
    'learning_rate': 0.08,  #best 0.08
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7, #best 0.7
    'verbosity':-1,
}    

In [5]:
import xgboost as xgb

In [7]:
dtrain

In [20]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    

    

### 本地CV

In [21]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.732724 + 0.00693564
[10]	cv_agg's res: 0.743003 + 0.00941347
[15]	cv_agg's res: 0.74606 + 0.00507338
[20]	cv_agg's res: 0.746784 + 0.00517028
[25]	cv_agg's res: 0.753843 + 0.0122167
[30]	cv_agg's res: 0.759055 + 0.0147335
[35]	cv_agg's res: 0.759338 + 0.0174121
[40]	cv_agg's res: 0.762528 + 0.0135725
[45]	cv_agg's res: 0.761319 + 0.0115971
[50]	cv_agg's res: 0.764783 + 0.0105879
[55]	cv_agg's res: 0.764508 + 0.0133178
[60]	cv_agg's res: 0.764239 + 0.0117978
[65]	cv_agg's res: 0.765524 + 0.0116594
[70]	cv_agg's res: 0.767698 + 0.0126035
[75]	cv_agg's res: 0.767827 + 0.0102039
[80]	cv_agg's res: 0.765357 + 0.00983224
[85]	cv_agg's res: 0.767948 + 0.00910029
[90]	cv_agg's res: 0.767155 + 0.0104562
[95]	cv_agg's res: 0.767164 + 0.0089063
[100]	cv_agg's res: 0.770738 + 0.00586505
[105]	cv_agg's res: 0.770073 + 0.00445654
[110]	cv_agg's res: 0.770056 + 0.00543194
[115]	cv_agg's res: 0.771595 + 0.00574698
[120]	cv_agg's res: 0.773584 + 0.00264619
[125]	cv_agg's res: 0.7750

{'res-mean': [0.71427936415198812,
  0.70063497222421045,
  0.72216304452747193,
  0.7292367128860836,
  0.73272438066569923,
  0.73693390962624938,
  0.73959018237545093,
  0.74472854996211524,
  0.74128415835077499,
  0.74300261822855829,
  0.74389029720957467,
  0.74638193600460712,
  0.74746594880507422,
  0.746822253600537,
  0.74606039050997419,
  0.75054097402601938,
  0.75117730203979438,
  0.75096250880398407,
  0.75114290411928264,
  0.74678447056004293,
  0.75456746915149797,
  0.75310476028216844,
  0.75486642697510276,
  0.75574400222261529,
  0.75384349287985009,
  0.75421303398079775,
  0.7550085790997455,
  0.75722119294876278,
  0.7586650396159057,
  0.75905504598403972,
  0.75983986801266179,
  0.76048292905960457,
  0.75919504412232197,
  0.75958089305804999,
  0.75933757970788929,
  0.75862908495464598,
  0.75967376533444575,
  0.7593983362358091,
  0.76230930874822977,
  0.76252806398873541,
  0.76400321940629823,
  0.76393558940495998,
  0.76191158023642613,
  0.7

## 训练

In [22]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.883107
[10]	training's res: 0.901904
[15]	training's res: 0.91778
[20]	training's res: 0.929665
[25]	training's res: 0.941872
[30]	training's res: 0.950961
[35]	training's res: 0.963817
[40]	training's res: 0.973273
[45]	training's res: 0.980423
[50]	training's res: 0.984904
[55]	training's res: 0.989673
[60]	training's res: 0.993339
[65]	training's res: 0.996377
[70]	training's res: 0.997752
[75]	training's res: 0.999106
[80]	training's res: 0.999554
[85]	training's res: 0.999555
[90]	training's res: 1
[95]	training's res: 1
[100]	training's res: 1
[105]	training's res: 1
[110]	training's res: 1
[115]	training's res: 1
[120]	training's res: 1
[125]	training's res: 1
[130]	training's res: 1
[135]	training's res: 1
[140]	training's res: 1
[145]	training's res: 1
[150]	training's res: 1
[155]	training's res: 1
[160]	training's res: 1
[165]	training's res: 1
[170]	training's res: 1
[175]	training's res: 1
[180]	training's res: 1
[185]	training's res: 1
[190]	training

### 预测

In [23]:
pred=model.predict(test.drop(['uid'],axis=1))

In [24]:
res =pd.DataFrame({'uid':test.uid,'label':pred})


In [25]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.5 else 0)
res.label = res.label.map(lambda x: int(x))

In [26]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])